# Homework 4: Systems of Equations

In [ ]:
import Pkg; Pkg.add("Plots");
using LinearAlgebra; using Plots; default(fmt=:png);
BLAS.set_num_threads(1);

In [ ]:
bigspy(A; kwargs...) = spy(A, m=4; kwargs...)

## 1. (T&B Exercise 20.2)

Suppose $A \in \mathbb{C}^{m \times m}$ has an LU factorization (by T&B Exercise 20.1, this is true if and only if for each $1 \leq k \leq m$ the upper-left $k \times k$ block $A_{1:k,1:k}$ is nonsingular) and is banded with bandwidth $2p + 1$, i.e., $a_{ij} = 0$ for $|i - j| > p$.

### 1.(a) [5%]

Create a random $20 \times 20$ matrix $A$ with bandwidth $2p + 1 = 5$, i.e. $p=2$, and compute its LU factorization $A = LU$ (without pivoting: see the [documentation](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.lu) to determine how to turn off pivoting in `lu`).  Plot the matrices $A$, $L$ and $U$ using the `bigspy` command.

In [ ]:
# your code here

### 1.(b) [5%]

Without adding the plots here, try the same for different values of $p$ until you see a pattern in the sparsity patterns of $L$ and $U$.  State what that pattern is and prove that it holds for general $p$.

YOUR ANSWER HERE

## 2. (T&B Exercise 21.2)

### 2.(a) [5%]

Repeat 1.(a), but this time with partial pivoting, and generate the same three plots.

In [ ]:
# your code here

### 2.(b) [5%]

What can be said about the bandedness of $U$?  Specifically, you should determine $p_U$ such that the bandwidth of $U$ is at most $2p_U + 1$ as a function of $p$. (Again, you may want to test your result on several random inputs until you see a pattern.) Prove your result.

YOUR ANSWER HERE

### 2.(c) [5%]

What is the most nonzeros entries $n_L$ that are in any column of $L$? Prove your result.

YOUR ANSWER HERE

## 3. (T&B Exercise 20.4) [10%]

Write the LU factorization algorithm (without pivoting) using just one explicit `for` loop.  Inside this loop, $U$ will be updated at each step by a certain rank-one outer product.  This "outer product" form of Gaussian elimination may be a better starting point if one wants to optimize computer performance.

In [ ]:
function lu_one_loop(A)
    """
    Compute the LU factorization of A using one explicit for loop.
    
    Arguments:
    
    - `A`: A square matrix
    
    Returns:
    
    - `L`: A unit lower triangular matrix
    
    - `U`: An upper triangular matrix such that A = L*U
    """
    # your code here
    return L, U
end

In [ ]:
function generate_random_lu_data(T, m)
    """
    Generate a random matrix that is guaranteed to have an LU factorization without pivoting.
    
    Arguments:
    
    - `T`: A datatype
    - `m`: A positive integer
    
    Returns:
    
    - `A`: an m × m matrix that has an LU factorization
    """
    
    A = randn(T, m, m)
    col_sum = abs.(A)' * ones(m)
    signs = sign.(diag(A))
    A += 1.1 * diagm(col_sum .* signs)
    return A
end

In [ ]:
# Test on real inputs
m = 15
A = generate_random_lu_data(Float64, m);

In [ ]:
# Q3: Run `lu_one_loop` on real inputs
L, U = lu_one_loop(A);

In [ ]:
# Q3: Real L is lower triangular
@assert(norm(L - LowerTriangular(L)) == 0)

In [ ]:
# Q3: Real L is unit lower triangular
@assert(norm(diag(L) - ones(m)) == 0)

In [ ]:
# Q3: Real U is upper triangular
@assert(norm(U - UpperTriangular(U)) == 0)

In [ ]:
# Q3: Real LU is close to A
ρ_est = sqrt(m) # average case observed growth factor
@assert(norm(A - L*U) <= 100 * eps() * ρ_est * norm(A))

In [ ]:
# Test on complex inputs
m = 15
A = generate_random_lu_data(Complex{Float64}, m);

In [ ]:
# Q3: Run `lu_one_loop` on complex inputs
L, U = lu_one_loop(A);

In [ ]:
# Q3: Complex L is lower triangular
@assert(norm(L - LowerTriangular(L)) == 0)

In [ ]:
# Q3: Complex L is unit lower triangular
@assert(norm(diag(L) - ones(m)) == 0)

In [ ]:
# Q3: Complex U is upper triangular
@assert(norm(U - UpperTriangular(U)) == 0)

In [ ]:
# Q3: Complex LU is close to A
ρ_est = sqrt(m) # average case observed growth factor
@assert(norm(A - L*U) <= 100 * eps() * ρ_est * norm(A))

## 4. (T&B Exercise 21.6) [10%]

You'll notice that `generate_random_lu_data` first creates a matrix and then adds a multiple of the column sum
$\sum_{i} |a_{ij}|$ to the diagonal $a_{jj}$, and that the result should have an LU factorization without pivoting.

Suppose $A \in \mathbb{C}^{m \times m}$ is _strictly column diagonally dominant_, which means that for each $k$,

$$|a_{kk}| > \sum_{j \neq k} |a_{jk}|.$$

Show that if Gaussian elimination with partial pivoting is applied to $A$, no row interchanges take place.

YOUR ANSWER HERE

## 5. (T&B Exercise 22.2)

### 5.(a) [5%]

Write a function that for a given $m$ generate an $m \times m$ matrix $A$ for which the backward error of
pivoted LU factorization, $\tilde{L}\tilde{U} = \tilde{P} A + \delta A$ grows like the pessimistic bound

$$\frac{\max_{ij} |(\delta A)_{ij}|}{\max_{ij} |A_{ij}|} = O(2^{m-1} \epsilon_{\text{machine}}).$$

In particular, your method should generate matrices such that the following bounds hold:

| $m$ | $\max_{ij} |(\delta A)_{ij}| / \max_{ij} |A_{ij}|$ |
| :-: | :-: |
| $30$  | $\geq 2^{20} \epsilon_{\text{machine}}$ |
| $40$  | $\geq 2^{30} \epsilon_{\text{machine}}$ |
| $50$  | $\geq 2^{40} \epsilon_{\text{machine}}$ |

In [ ]:
function catastrophic_growth_factor_matrix(m)
    """
    Construct a matrix with nearly pessimistic growth factor
    
    Arguments:
    
    - `m`: An integer
    
    Returns:
    
    - `A`: An m × m matrix whose growth factor (computed with julia's `lu`) is close
           to the upper bound 2ᵐ⁻¹.
    """
    # your code here
    return A
end

In [ ]:
# Q5: test m = 30
m = 30
A = catastrophic_growth_factor_matrix(m)
f = lu(A)
rel_err = norm(f.P * A - f.L * f.U, Inf) / norm(A, Inf)
@assert rel_err >= eps() * 2.0 ^ (m - 10)

In [ ]:
# Q5: test m = 40
m = 40
A = catastrophic_growth_factor_matrix(m)
f = lu(A)
rel_err = norm(f.P * A - f.L * f.U, Inf) / norm(A, Inf)
@assert rel_err >= eps() * 2.0 ^ (m - 10)

In [ ]:
# Q5: test m = 50
m = 50
A = catastrophic_growth_factor_matrix(m)
f = lu(A)
rel_err = norm(f.P * A - f.L * f.U, Inf) / norm(A, Inf)
@assert rel_err >= eps() * 2.0 ^ (m - 10)

### 5.(b) [5%]

Make a plot of the backward relative errors $\max_{ij} |(\delta A)_{ij}| / \max_{ij} |A_{ij}|$ for your matrices for values $m=10, 11, \dots, 60$.  Make $m$ the x-axis and the relative error the y-axs, and make the y-axis a logarithmic scale.
Add a trend line for $2^{m-1} \epsilon_{\text{machine}}$ to your plot to show what the asymptotic growth looks like in the worst case.

In [ ]:
# your code here

## 6. (T&B Exercise 22.4)

### 6.(a) [5%]

Suppose $PA = LU$ (LU factorization with partial pivoting) and $A = QR$ (QR factorization).  Describe a relationship between the last row of $L^{-1}$ and the last column of $Q$, and prove why this relationship is so.

YOUR ANSWER HERE

### 6.(b) [5%]

Show that if $A$ is random in the sense of having independent, normally distributed entries, then its column spaces are randomly oriented, so that in particular, the last column of $Q$ is a random unit vector.

YOUR ANSWER HERE

### 6.(c) [5%]

Combine the results of (a) and (b) to make a statement about the final row of $L^{-1}$ in Gaussian elimination applied to a random matrix $A$.

YOUR ANSWER HERE

## 7.

### 7.(a) [5%]

Give an example of a $3 \times 3$ matrix $A$ in block form

$$
A = \begin{bmatrix} F & B \\ B^* & 0 \end{bmatrix},
$$

where:

- $F$ is a $2 \times 2$ symmetric positive definite matrix and $\kappa(F) \geq 10^{10}$, and
- $\kappa(A) < 2$.

YOUR ANSWER HERE

## 7.(b) [5%]

Consider the problem of solving the system of equations

$$
\begin{bmatrix}
C & B \\ B^* & 0
\end{bmatrix}
\begin{bmatrix}
x \\ y
\end{bmatrix}
=
\begin{bmatrix}
f \\ g
\end{bmatrix},
$$

where $C$ is symmetric positive definite and $B \in \mathbb{C}^{m \times n}$ ($m \geq n$) has full rank.  Suppose we solve this system in the following steps:

1. Compute the Cholesky factorization $C = LL^*$.
2. Compute $w = g - B^*C^{-1} f = g - B^* L^{-*} L^{-1} f$.
3. Compute the matrix $S= B^* C^{-1} B = B^* L^{-*} L^{-1} B$.
4. Compute the Cholesky factorization $G G^* = S$.
5. Compute $y = -S^{-1} w = - G^{-*} G^{-1} w$.
6. Compute $z = f - B y$.
7. Compute $x = C^{-1} z = L^{-*} L^{-1}z$.

Why does part $A$ suggest that this is method may be unstable?

YOUR ANSWER HERE

### 7.(c) [10%]

Now suppose we consider only problems where $g = 0$,

$$
\begin{bmatrix}
C & B \\ B^* & 0
\end{bmatrix}
\begin{bmatrix}
x \\ y
\end{bmatrix}
=
\begin{bmatrix}
f \\ 0
\end{bmatrix},
$$

so that $y = (B^* C^{-1} B)^{-1} B^* C^{-1} f.$

Prove that there is a bound $\psi_B$ such that

$$\|(B^* C^{-1} B)^{-1} B^* C^{-1}\| \leq \psi_B, \quad \text{for all symmetric positive definite }C.$$

This suggests that the condition number of $\kappa_{C \to y}$ is independent of $C$.

_Hint:_ Use two SVDs: one of $B$, one of $B^* C^{-1}$.

YOUR ANSWER HERE

## 8. 

### 8.(a) [6%]

Write an implementation of a rank-one Cholesky update: assuming $L L^* = A$ is a Cholesky factorization of $A$,
compute $H H^* = A + u u^*$, the Cholesky factorization of the rank-one update of $A$.

Unlike a Cholesky factorization from scratch, your implementation should use $O(m^2)$ operations.

You are welcome to consult [existing implementations](https://en.wikipedia.org/wiki/Cholesky_decomposition#Updating_the_decomposition), but your implementation will have to be valid for complex-valued $A$ and $u$.

In [ ]:
function cholesky_update(L, u)
    """
    Update a Cholesky factorization by a rank one update,
    
        H H' = L L' + u u'
    
    Arguments:
    
    - `L`: An existing lower-triangular Cholesky factor
    
    - `u`: The vector for the rank-one update
    
    Returns:
    
    - `H`: the lower-triangular Cholesky factor of `L L' + u u'`
    
    """
    # your code here
    return H
end

In [ ]:
# Q8.(a): Run cholesky_update on real input
m = 50
B = randn(m,m)
A = B' * B
L = cholesky(A).L
u = randn(m)
Au = A + u*u'
H = cholesky_update(L, u);

In [ ]:
# Q8.(a): Real Cholesky factor is lower triangular
@assert(norm(H - LowerTriangular(H)) == 0)

In [ ]:
# Q8.(a): Real Cholesky factor has small backward error
@assert(norm(Au - H * H') / norm(Au) <= eps() * 100)

In [ ]:
# Q8.(a): Run cholesky_update on complex input
m = 50
B = randn(Complex{Float64}, m,m)
A = B' * B
L = cholesky(A).L
u = randn(Complex{Float64}, m)
Au = A + u*u'
H = cholesky_update(L, u);

In [ ]:
# Q8.(a): Complex Cholesky factor is lower triangular
@assert(norm(H - LowerTriangular(H)) == 0)

In [ ]:
# Q8.(a): Complex Cholesky factor has small backward error
@assert(norm(Au - H * H') / norm(Au) <= eps() * 100)

### 8.(b) [4%]

Suppose that we only want to solve the system $(A + u u^*) x = b$.

Again assume that $L L^* = A$ is a Cholesky factorization of $A$, but instead of updating the factorization, write a function that solves the system using the [Woodbury formula](https://en.wikipedia.org/wiki/Woodbury_matrix_identity).

In [ ]:
function solve_cholesky_update(L, u, b)
    """
    Solve (L L' + u u') x = b
    
    Arguments:
    
    - `L`: An existing lower-triangular Cholesky factor
    
    - `u`: The vector for the rank-one update
    
    Returns:
    
    - `x` : the solution of (L L' + u u') x = b
    """
    # your code here
    return x
end

In [ ]:
# Q8.(b): Run solve_cholesky_update on real input
m = 50
B = randn(m,m)
A = B' * B
L = cholesky(A).L
u = randn(m)
b = randn(m)
Au = A + u*u'
x = solve_cholesky_update(L, u, b);

In [ ]:
# Q8.(b): Real cholesky update solve has small backward error
@assert(norm(Au * x - b) / norm(b) <= eps() * 1.e5)

In [ ]:
# Q8.(b): Run solve_cholesky_update on complex input
m = 50
B = randn(Complex{Float64}, m,m)
A = B' * B
L = cholesky(A).L
u = randn(Complex{Float64}, m)
b = randn(Complex{Float64}, m)
Au = A + u*u'
x = solve_cholesky_update(L, u, b);

In [ ]:
# Q8.(b): Complex cholesky update solve has small backward error
@assert(norm(Au * x - b) / norm(b) <= eps() * 1.e5)